In [1]:
import pandas as pd
import numpy as np

In [6]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

### Base Version

In [7]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [8]:
%%timeit
target_mean_v1(data,'y','x')

1.77 s ± 69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Without using groupby

In [19]:
def target_mean_v2(data,y_name, x_name):
    sum_dict = dict()
    count_dict = dict()
    # calculate total sum(count) of y_label 
    for i,val in enumerate(data[x_name].values):
        sum_dict[val] = sum_dict.get(val,0)+ data.loc[i,y_name]
        count_dict[val] = count_dict.get(val,0)+1
    # calculate the final result
    result = np.zeros(data.shape[0])
    for i in range(len(data)):
        result[i] = (sum_dict[data.loc[i,x_name]]-data.loc[i,y_name])/(count_dict[data.loc[i,x_name]]-1)
    return result   

In [20]:
%%timeit
target_mean_v2(data,'y','x')

10.9 ms ± 476 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
def target_mean_v3(data,y_name, x_name):
    sum_dict = dict()
    count_dict = dict()
    # calculate total sum(count) of y_label 
    for i,val in enumerate(data[x_name].values):
        sum_dict[val] = sum_dict.get(val,0)+ data.loc[i,y_name]
        count_dict[val] = count_dict.get(val,0)+1
    # calculate the final result
    result = np.zeros(data.shape[0])
    for i in range(len(data)):
        tmp_x = data.loc[i,x_name]
        result[i] = (sum_dict[tmp_x]-data.loc[i,y_name])/(count_dict[tmp_x]-1)
    return result 

In [22]:
%%timeit
target_mean_v3(data,'y','x')

8.37 ms ± 435 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


*异常值处理*：分子为0时